# Disaster Tweet Classification Using BERT

### Import Training Data

In [8]:
import pandas as pd

data = pd.read_csv("train.csv")
data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


### Explore training data

In [9]:
data.groupby('target').count()

,id,keyword,location,text
target,,,,
0,4342,4323,2884,4342
1,3271,3229,2196,3271


### Import BERT model and pre-processor

In [10]:
import tensorflow as tf
import tensorflow_hub as hub    
import tensorflow_text as text 

bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

### Build Model

In [11]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [12]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_2 (KerasLayer)     {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128)}                                                

In [16]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

### Generate train and test set

In [21]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data["text"],data["target"], stratify=data["target"])

### Train Model

In [22]:
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
179/179 [==============================] - 20s 113ms/step - loss: 0.5259 - accuracy: 0.7514 - precision: 0.7402 - recall: 0.6494
Epoch 2/10
179/179 [==============================] - 20s 113ms/step - loss: 0.5300 - accuracy: 0.7507 - precision: 0.7371 - recall: 0.6527
Epoch 3/10
179/179 [==============================] - 20s 112ms/step - loss: 0.5272 - accuracy: 0.7544 - precision: 0.7423 - recall: 0.6563
Epoch 4/10
179/179 [==============================] - 20s 112ms/step - loss: 0.5112 - accuracy: 0.7625 - precision: 0.7536 - recall: 0.6645
Epoch 5/10
179/179 [==============================] - 20s 112ms/step - loss: 0.5197 - accuracy: 0.7597 - precision: 0.7503 - recall: 0.6604
Epoch 6/10
179/179 [==============================] - 20s 113ms/step - loss: 0.5119 - accuracy: 0.7634 - precision: 0.7565 - recall: 0.6625
Epoch 7/10
179/179 [==============================] - 20s 113ms/step - loss: 0.5145 - accuracy: 0.7604 - precision: 0.7499 - recall: 0.6637
Epoch 8/10
179/179 [

### Evaluate Model

In [23]:
model.evaluate(x_test, y_test)

60/60 [==============================] - 8s 111ms/step - loss: 0.5031 - accuracy: 0.7610 - precision: 0.7065 - recall: 0.7592


[0.503096878528595, 0.7610294222831726, 0.7064846158027649, 0.7591686844825745]

In [25]:
y_predicted = model.predict(x_test)
y_predicted = y_predicted.flatten()

In [26]:
import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([0, 0, 1, ..., 1, 1, 1])

In [27]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test, y_predicted)
cm 

array([[828, 258],
       [197, 621]], dtype=int64)

In [28]:
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.81      0.76      0.78      1086
           1       0.71      0.76      0.73       818

    accuracy                           0.76      1904
   macro avg       0.76      0.76      0.76      1904
weighted avg       0.76      0.76      0.76      1904

